In [ ]:
# Install ffmpeg
!apt-get update
!apt-get install -y ffmpeg

# # Verify installation
# !ffmpeg -version
!pip install evaluate
!pip install jiwer
!pip install torchcodec
!pip install silero-vad

In [ ]:
!pip install --upgrade transformers accelerate

In [ ]:
import pandas as pd
import os

from transformers import WhisperForConditionalGeneration
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from transformers import WhisperFeatureExtractor
import torch
from transformers import  pipeline

import json
import evaluate
from peft import PeftModel

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
ROOT_FOLDER = '/content/drive/My Drive/Project/'

checkpoint_dir = os.path.join(ROOT_FOLDER, 'finetuned_model_files','Small_R32_lr_4_10-4' , "checkpoint-4185")
base_model_name = "small"

batchsize = 8

In [ ]:
# ROOT_FOLDER = ''
# checkpoint_dir = os.path.join(ROOT_FOLDER, 'finetuned_model_files','Small_R8_lr_3_10-4' , "checkpoint-4650")
# base_model_name = "small"

## Pretrained whisper-small Test validation res

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-" + base_model_name)
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-" + base_model_name,
    language="English",      # sets English as decoding language
    task="transcribe",        # task can be 'transcribe' or 'translate'
    feature_extractor_type="whisper",  # Explicitly set feature extractor type
    feature_size=80 # Set feature size to 80
)
model.generation_config.task = 'transcribe'
model.generation_config.forced_decoder_ids = None

# Attach LoRA weights
model = PeftModel.from_pretrained(model, checkpoint_dir)

# # Read JSON file
# ROOT_FOLDER = '.'
# with open(os.path.join(ROOT_FOLDER, 'data/Processed_Files', "Transcript.json"), "r", encoding="utf-8") as f:
#     transcript_json = json.load(f)


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Project')
from utils.Preprocessing_inference import inference
from transformers import pipeline

In [ ]:
asr_pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        torch_dtype=torch.float32,
        device=0 if torch.cuda.is_available() else -1
    )


In [ ]:
inf_class = inference(os.path.join(ROOT_FOLDER, 'data','Rec' , "20241208094232_9811608043.mp3") ,asr_pipe, batchsize )

In [ ]:
res = inf_class.Transcribe()

In [ ]:
print(res)